In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-11-11 20:01:40.193118


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Missing Artists From Albums

In [5]:
%load_ext autoreload
%autoreload
from albums import albums
from discogsBase import discogs

disc  = discogs()
albs  = albums(disc)
#toget = albs.findMissingArtistsFromAlbums()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


In [6]:
from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            print('\n{0}/{1}'.format(ia,len(refstoget)))
            retval = arts.downloadArtistURL(url, savename, debug=True)
            sleep(2)
        except:
            continue
            
    #print(item)
#len(togetmap)

ZeroDivisionError: division by zero

# Artists From Search

In [ ]:
arts.searchDiscogForArtist("Dodo And The Dodo's")

# Albums From Known People

In [7]:
data = getFile("/Volumes/Music/MusicDB/toget.p")

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
discdf = disc.getMasterDiscogsDB()
print(discdf.shape)
discdf = discdf[~discdf['Name'].isna()]
print(discdf.shape)

artists = list(discdf["Artist"])

_ = clock("Last Run")

In [ ]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
toget={}
known={}

for ia,artistName in enumerate(data):
    #artistID = artistName
    artistID = None
    if artistID is None:
        retval = getMusicData('Name', artistName)
        if retval is not None:
            artistID = retval.index.values[0]
    modValue  = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
    if toget.get(modValue) is None:
        toget[modValue] = {}
    toget[modValue][artistID] = True
        
    if ia % 100 == 0:
        print(ia,'/',len(data))

In [ ]:
for modVal in toget.keys():
    artistIDs = toget[modVal]
    print(modVal,'\t',len(artistIDs))
    dbdata = disc.getArtistsDBModValData(modVal)
    for artistID in artistIDs:
        print(' ',artistID)
        artistData = dbdata[artistID]
        albs.downloadAlbumFromArtistData(artistID=artistID, artistData=artistData, mediaTypes=None)

# Albums From All Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=1, mediaTypes=None, rand=True)

# Artists From Collections

In [9]:
%load_ext autoreload
%autoreload

from collection import Counter

refCounts = Counter(disc.getCollectionRefCountsData())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading data from /Volumes/Music/Discog/db/CollectionRefCounts.p
  --> This file is 27.1MB.
Loading /Volumes/Music/Discog/db/CollectionRefCounts.p


In [29]:
try:
    errFile = setFile(disc.getDiagnosticDir(), "CollectionErrorsX.json")
    errs = getFile(errFile)
except:
    errs = {}
    
print("Found {0} errors".format(len(errs)))
nNext = len(errs)

Found 0 errors


In [30]:
n = len(refCounts)
nFalse = 0
nNone  = 0
nTrue  = 0
artistIDs = disc.getArtistIDToRefData()

for i,(ref,cnts) in enumerate(refCounts.most_common()):
    if i % 2500 == 0:
        print("Processed {0}/{1}  (Errs={2})".format(i,n, len(errs)))
        
    if errs.get(ref) is not None:
        continue
        
    artistID = dutils.getArtistID(ref)
    if artistID is None:
        continue
    if artistIDs.get(artistID) is None:
        url      = arts.getArtistURL(ref)
        savename = arts.getArtistSavename(artistID)
        if isFile(savename):
            continue
            
        try:
            retval = arts.downloadArtistURL(url=url, savename=savename, parse=True, debug=True)
        except:
            retval = None
            
        if retval is None:
            nNone += 1
            errs[ref] = "None"
        if retval is False:
            nFalse += 1
            errs[ref] = "False"
            
        sleep(3)

    if (nNone % 100 == 0 or nFalse % 100 == 0) and (nNone > 0 and nFalse > 0):
        print("There have been {0} errors so far...".format(nNone))
        print("There have been {0} no saves so far...".format(nFalse))
        print("There have been {0} full saves so far...".format(nTrue))
        
        errFile = setFile(disc.getDiagnosticDir(), "CollectionErrors.json")
        saveFile(ifile=errFile, idata=errs)


            
    if len(errs) == nNext + 250:
        errFile = setFile(disc.getDiagnosticDir(), "CollectionErrors.json")
        saveFile(ifile=errFile, idata=errs)
        print("Breaking after 250 errors")
        break

Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 12.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Processed 0/1490722  (Errs=0)
Processed 2500/1490722  (Errs=0)
Processed 5000/1490722  (Errs=0)
Now Downloading in Artists(): https://www.discogs.com/artist/332344-Clarence-Brown?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/44/332344.p
Processed 7500/1490722  (Errs=1)
Processed 10000/1490722  (Errs=1)
Processed 12500/1490722  (Errs=1)
Processed 15000/1490722  (Errs=1)
Processed 17500/1490722  (Errs=1)
Processed 20000/1490722  (Errs=1)
Now Downloading in Artists(): https://www.discogs.com/artist/540538-Tlahoun-G%C3%83%C2%A8ss%C3%83%C2%A8ss%C3%83%C2%A8?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/38/540538.p
Now Downloading in Artists(): https://www.discogs.com/artist/488305-The-Benedictine-Monks-Of-Santo-Domingo-De-Silos?sort=year%2Casc&limit=500
                   Saving a

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-30-8212f2f97486>", line 35, in <module>
    sleep(3)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/cor

KeyboardInterrupt: 

In [24]:
len(errs)

500

In [25]:
disc.getDiagnosticDir()

'/Volumes/Music/Discog/diagnostic'